<a target="_blank" href="https://colab.research.google.com/github/sarthakrastogi/quality-prompts/blob/main/examples/few_shot_prompt_usage.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture
!pip install quality-prompts

In [2]:
import requests
import json
import os

from quality_prompts.prompt import QualityPrompt
from quality_prompts.exemplars import ExemplarStore, Exemplar
from quality_prompts.utils.llm import get_embedding

In [3]:
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY_HERE"

### Initialise sample exemplars for use in few-shot prompt

In [4]:
url = "https://github.com/sarthakrastogi/quality-prompts/raw/main/examples/sample_exemplars.json"
response = requests.get(url)
sample_exemplars = json.loads(response.text)

exemplars = [Exemplar(input=e['input'], label=str(e['label']), input_embedding=get_embedding(e['input'])) for e in sample_exemplars]
exemplar_store = ExemplarStore(exemplars=exemplars)

### Define your prompt's components and initialise it

In [5]:
directive = """You are given a document and your task is to create a knowledge graph from it."""

additional_information = """
In the knowledge graph, entities such as people, places, objects, institutions, topics, ideas, etc. are represented as nodes.
Whereas the relationships and actions between them are represented as edges.
"""

output_formatting = """
You will respond with a knowledge graph in the given JSON format:

[
    {"entity" : "Entity_name", "connections" : [
        {"entity" : "Connected_entity_1", "relationship" : "Relationship_with_connected_entity_1},
        {"entity" : "Connected_entity_2", "relationship" : "Relationship_with_connected_entity_2},
        ]
    },
    {"entity" : "Entity_name", "connections" : [
        {"entity" : "Connected_entity_1", "relationship" : "Relationship_with_connected_entity_1},
        {"entity" : "Connected_entity_2", "relationship" : "Relationship_with_connected_entity_2},
        ]
    },
]

You must strictly respond in the given JSON format or your response will not be parsed correctly!
"""

prompt = QualityPrompt(
                        directive=directive,
                        additional_information=additional_information,
                        output_formatting=output_formatting,
                        exemplar_store=exemplar_store
                       )

In [6]:
input_text = "list the disorders included in cvd"

### Apply few-shot prompting

This searches through your set of exemplars and uses kNN to search the most relevant exemplars to be included in context.

In [7]:
prompt.few_shot(input_text=input_text, n_shots=2)

In [8]:
# check the relevant exemplars searched for the user's query

for exemplar in prompt.few_shot_examples:
    print(exemplar.input, exemplar.label)

Cardiovascular disease (CVD) encompasses a spectrum of disorders involving the heart and vasculature, prominently including atherosclerosis, characterized by endothelial dysfunction and the accumulation of lipid-laden plaques. These pathophysiological processes often precipitate myocardial infarction and cerebrovascular accidents, arising from the rupture of vulnerable plaques and subsequent thrombogenesis. [{'entity': 'cardiovascular disease (cvd)', 'connections': [{'entity': 'heart', 'relationship': 'involves'}, {'entity': 'vasculature', 'relationship': 'involves'}, {'entity': 'atherosclerosis', 'relationship': 'associated disorder'}, {'entity': 'endothelial dysfunction', 'relationship': 'characteristic feature'}, {'entity': 'lipid-laden plaques', 'relationship': 'associated feature'}, {'entity': 'myocardial infarction', 'relationship': 'common complication'}, {'entity': 'cerebrovascular accidents', 'relationship': 'common complication'}, {'entity': 'plaque rupture', 'relationship': 

### Apply other prompting techniques

In [9]:
prompt.system2attenton(input_text=input_text)

### Finally compile your prompt for use in an LLM

In [10]:
print(prompt.compile())

You are given a document and your task is to create a knowledge graph from it.
        In the knowledge graph, entities such as people, places, objects, institutions, topics, ideas, etc. are represented as nodes. Whereas the relationships and actions between them are represented as edges.
        Example input: Cardiovascular disease (CVD) encompasses a spectrum of disorders involving the heart and vasculature, prominently including atherosclerosis, characterized by endothelial dysfunction and the accumulation of lipid-laden plaques. These pathophysiological processes often precipitate myocardial infarction and cerebrovascular accidents, arising from the rupture of vulnerable plaques and subsequent thrombogenesis.
Example output: [{'entity': 'cardiovascular disease (cvd)', 'connections': [{'entity': 'heart', 'relationship': 'involves'}, {'entity': 'vasculature', 'relationship': 'involves'}, {'entity': 'atherosclerosis', 'relationship': 'associated disorder'}, {'entity': 'endothelial dy